In [ ]:
import numpy as np
import cvxopt as cvx
import control as con
from numpy import linalg as LA

In [ ]:
from cvxpy import *

# Create two scalar optimization variables.
x = Variable()
y = Variable()

# Create two constraints.
constraints = [x + y == 1,
               x - y >= 1]

# Form objective.
obj = Minimize(square(x - y))

# Form and solve problem.
prob = Problem(obj, constraints)
prob.solve()  # Returns the optimal value.
print "status:", prob.status
print "optimal value", prob.value
print "optimal var", x.value, y.value

In [ ]:
print installed_solvers()

In [ ]:
### Seite 37 (im Text zwischen (4.12) und (4.13))
def _N(n):
    return np.diag([p for p in xrange(-n, 0, 1)])

### Seite 55; (4.64)
def _M(n):
    return np.diag([p for p in xrange(0, n, 1)])

### Seite 55; (4.64)
def _P(l, k, n):
    I = np.eye(n)
    Mn = _M(n)
    P = I
    if k == 0:
        pass # P=I
    else:
        for q in xrange(0, k, 1):
            P = P * ((l-q)*I + Mn)
    return cvx.matrix(P)

In [ ]:
###################################################
# Boris Diss Hydraulisches Positioniersystem S.61 #
###################################################
A = np.matrix([[0,      1,        0],
               [0, -0.126,     83.5],
               [0,  -6.89, -0.00707]])
 
b = np.matrix([[0],[0],[-55.51]])
c = np.matrix([1, 0, 0])

print "Poles:\n", np.matrix(con.matlab.pole(con.matlab.ss(A, b, c, 0))).T

u_max = 1
n = len(b)

X0 = [np.matrix([-13.38, -10.7, -2.58]).T,
      np.matrix([-13.38, -10.7,  2.58]).T,
      np.matrix([-13.38,  10.7, -2.58]).T,
      np.matrix([-13.38,  10.7,  2.58]).T,
      np.matrix([ 13.38, -10.7, -2.58]).T,
      np.matrix([ 13.38, -10.7,  2.58]).T,
      np.matrix([ 13.38,  10.7, -2.58]).T,
      np.matrix([ 13.38,  10.7,  2.58]).T]

x_max = [13.38, 10.7, 2.58]

#print "A:\n", A
#print "a:\n", a
#print "b:\n", b
#print "c:\n", c
p_min = 0.2
mu_star = 1.5
zeta = 2.5
zeta_star = 1.5

k0 = 1e-3*np.matrix([-24.23, -0.2122, -10.5]).T
k1 = 1e-3*np.matrix([-40.13, -0.1396, -15.10]).T
k0_star = 1e-3*np.matrix([-11.96, -0.1111, -5.18]).T
k1_star = 1e-3*np.matrix([-83.35, -0.0477, -33.86]).T

##### Entwurf parameter #####
beta=2 # beta >=1

In [ ]:
def someparameters():
    k0 = np.matrix([[ 0.00702601],
     [-0.79417547],
     [-0.01583519]])

    k1 = np.matrix([[ 0.00044966],
     [-0.02280205],
     [-0.00489574]])

    k0_star = np.matrix([[ 0.00151984],
     [-0.2060622 ],
     [-0.00826113]])

    k1_star = np.matrix([[ 0.00045032],
     [-0.02262498],
     [-0.00470835]])


In [ ]:
# Test initial values for following problem 
m = 1
g_val = 2.1 # Bisection parameter

In [ ]:
### Design Sättigungsregler mittels konvexer Hülle (A.3)

# Variables (Convex)
#  Name in A.3 | Name in Program
#    Q  = P^⁻1 |   = Q
#    z  = Ql   |   = z0
#    z* = Ql*  |   = z1

# Variables (Quasi convex)
#    gamma     |   = g

# Parameter
#    mu        |   = m (Designparameter)
#    X0        |   = X0 = [x0,...]
#    A         |   = A
#    b         |   = b

# Initialize
import cvxpy
n = len(b) # get dim of system

# Define Variables
#Q  = cvxpy.Semidef(n) #symmetric and positive semidefinite
Q  = cvxpy.Variable(n, n) # Semidef could go as an additional constraint as well, thereby no need fo Q to be symmetric
const_sdQ = (Q >> 0) # semidef but not (necessarily) symmetric

z0 = cvxpy.Variable(n)
z1 = cvxpy.Variable(n)

# Bisection parameter
g = cvxpy.Parameter(sign='positive')
g.value = g_val # TODO set by bisection loop (function?)

# Define Constraints
# (A.10)
const_A10 = [cvxpy.bmat([[Q,       X0[i]],
                         [X0[i].T, 1    ]]) >> 0
                            for i in range(0, len(X0))]

# (A.11)
const_A11 = cvxpy.bmat([[Q,  z0],
                        [z0.T, 1 ]]) >> 0

# (A.12)
const_A12 = cvxpy.bmat([[Q,    z1  ],
                        [z1.T,   m**2]]) >> 0

# (A.13)
const_A13 = Q*A + A.T*Q - b*z0.T - z0*b.T << 0 # This constraint is strict definit

# (A.14)
const_A14 = Q*A + A.T*Q - b*z1.T - z1*b.T << -2*g*Q # This constraint is strict definit

constraints.append(const_sdQ)
constraints.extend(const_A10) ##!! Beware of the "extend" if input is array
constraints.append(const_A12)
constraints.append(const_A13)
constraints.append(const_A14)

# Define Objective (Generalized eigenvalue problem?)
#obj = cvxpy.Maximize(g)
# Feasibility for bisection:
obj = cvxpy.Minimize(0)

# Form and solve problem.
prob = cvxpy.Problem(obj, constraints)
prob.solve(solver=cvxpy.SCS, verbose=False, max_iters=2500)  # Returns the optimal value.
#prob.solve()  # Returns the optimal value.

prob.status

In [ ]:
## Example bisection code (MATLAB)
## https://see.stanford.edu/materials/lsocoee364a/hw6sol.pdf (p.3)

## Example bisection code
## https://www.coursehero.com/file/p3eovvo/To-solve-the-problem-we-can-use-a-bisection-method-solving-an-LP-feasibility/

import numpy as np
import matplotlib.pyplot as plt
import cvxpy as cvx

k = 201
t = -3 + 6 * np.arange(k) / (k - 1)
y = np.exp(t)

Tpowers = np.vstack((np.ones(k), t, t**2)).T

a = cvx.Variable(3)
b = cvx.Variable(2)
gamma = cvx.Parameter(sign='positive')
lhs = cvx.abs(Tpowers * a - (y[:, np.newaxis] * Tpowers) * cvx.vstack(1, b))
rhs = gamma * Tpowers * cvx.vstack(1, b)
problem = cvx.Problem(cvx.Minimize(0), [lhs <= rhs])

l, u = 0, np.exp(3) # initial upper and lower bounds
bisection_tol = 1e-3 # bisection tolerance
while u - l >= bisection_tol:
    gamma.value = (l + u) / 2
    # solve the feasibility problem
    problem.solve()
    if problem.status == 'optimal':
        u = gamma.value
        a_opt = a.value
        b_opt = b.value
        objval_opt = gamma.value
    else:
        l = gamma.value
        
y_fit = (Tpowers * a_opt / (Tpowers * np.vstack((1, b_opt)))).A1

plt.figure()
plt.plot(t, y, 'b', t, y_fit, 'r+')
plt.xlabel('t')
plt.ylabel('y')
plt.show()

plt.figure()
plt.plot(t, y_fit - y)
plt.xlabel('t')
plt.ylabel('err')
plt.show()

In [ ]:
# A.4
# Transformationen

In [ ]:
import cvxpy

R0 = cvxpy.Variable(n,n)
R1 = cvxpy.Variable(n,n)

G = cvxpy.Variable(n,n) #skew
G_star = cvxpy.Variable(n, n) # skew

# D = cvxpy.Symmetric(n)
# D_star = cvxpy.Symmetric(n)
D = cvxpy.Variable(n, n) #sym
D_star = cvxpy.Variable(n, n) #sym

constraints = [D                 == D.T, # sym
               D_star            == D_star.T, # sym
               G + G.T           == 0, # skew
               G_star + G_star.T == 0] # skew


# Parameter
alpha = 1-p_min
beta = 1+p_min
    
A0 = A-b*k0.T
A1 = A-b*k1.T

S0 = A1.T*R1+R1*A1
S1 = A0.T*R1+R1*A0 + A1.T*R0+R0*A1
S2 = A0.T*R0+R0*A0

A0_star = A-b*k0_star.T
A1_star = A-b*k1_star.T

S0 = A1.T*R1+R1*A1
S1 = A0.T*R1+R1*A0 + A1.T*R0+R0*A1
S2 = A0.T*R0+R0*A0

S0_star = A1_star.T*R1+R1*A1_star
S1_star = A0_star.T*R1+R1*A0_star + A1_star.T*R0+R0*A1_star
S2_star = A0_star.T*R0+R0*A0_star

# Create two constraints.
const4_39ab = [R0 >> 0,
               R0 + R1 >> 0]

const4_39c = cvxpy.bmat([[-D , G],
                         [G.T, D]]) - \
             cvxpy.bmat([[S0 + 0.5*beta*S1 + 0.25*beta**2*S2, 0.25*alpha*(S1 + beta*S2)],
                         [0.25*alpha*(S1 + beta*S2),          0.25*alpha**2*S2]]) >> 0

const4_39d = [cvxpy.bmat([[1, X0[i].T],
                          [X0[i], R0+R1]]) >> 0
                              for i in range(0, len(X0))]

const4_39e = cvxpy.bmat([[1,    k0.T],
                         [k0,   R0]]) + \
             1/p_min * cvxpy.bmat([[0,  k1.T],
                                  [k1, R1]]) >> 0

const4_39f = cvxpy.bmat([[1,    k0.T],
                         [k0,   R0]]) + \
             cvxpy.bmat([[0,  k1.T],
                         [k1, R1]]) >> 0

const4_39g = cvxpy.bmat([[-D_star , G_star],
                         [G_star.T, D_star]]) - \
             cvxpy.bmat([[S0_star + 0.5*beta*S1_star + 0.25*beta**2*S2_star, 0.25*alpha*(S1_star + beta*S2_star)],
                         [0.25*alpha*(S1_star + beta*S2_star),          0.25*alpha**2*S2_star]]) >> 0

constraints.extend(const4_39ab) ##!! Beware of the "extend" if input is array
constraints.append(const4_39c)
constraints.extend(const4_39d) ##!! Beware of the "extend" if input is array
constraints.append(const4_39e)
constraints.append(const4_39f)
constraints.append(const4_39g)

# Form objective.
obj = cvxpy.Minimize(cvxpy.trace(R0+1/p_min*R1))

# Form and solve problem.
prob = cvxpy.Problem(obj, constraints)
prob.solve(solver=cvxpy.SCS, verbose=True, max_iters=2500)  # Returns the optimal value.
print "status:", prob.status
print "optimal value", prob.value
print "R0", R0.value
LA.cholesky(R0.value)
print "R1", R1.value
LA.cholesky(R1.value + R0.value)
print "ok"